In [ ]:
# Dashboard Komentare
# Das machen wir:
# Karte (Output) mit Slider (Input) (Raphi)
# Rangliste der Länder (Output) 
# Liniendiagramm (Output) fürs Vergleichen von zwei Länder mit Dropdown (Input) (aus Aufgabe 3)
# Balkendiagramm mehrere Länder (Output) (aus Aufgabe 3)


In [ ]:
# Importe
import pandas as pd
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output, callback
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

# Dateipfad
url = "https://raw.githubusercontent.com/hon3ypi/DashboardDesignFS23/main/newDataset.csv"

# Daten aus CSV-Datei laden
datafile = pd.read_csv(url)

# Daten für die ausgewählten Länder und Jahre filtern
countries = ['Switzerland', 'Germany', 'France', 'Austria', 'Italy', 'Liechtenstein']
filtComb = datafile.query('GeoAreaName in @countries')

# Daten für alle Länder
allCount = datafile['GeoAreaName'].unique()

# Diagramm für Tab 1
scatfig = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')
hisfig = px.histogram(filtComb, x="Year", y="Value", color='GeoAreaName', barmode='group')

# Web-App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Bereinigter Datensatz
data = pd.read_csv('https://raw.githubusercontent.com/monsieurmaussion/fhgr-dade/main/woman-leadership_bereinigt-neu.csv')

# Datensatz nach Jahr sortiert für animation_frame
data = data.sort_values(by="Year", ascending=True)

# Daten für alle Länder
allCountry = data['Country Name'].unique()

# Plotly choropleth map
hover_data = ["Country Name", "Year", "Value"]
mapfig = px.choropleth(data_frame=data,
                       color_continuous_scale=px.colors.sequential.Purples,
                       color="Value",
                       hover_data=hover_data,
                       locations='Country Code',
                       animation_frame='Year')

mapfig.update_layout(
    autosize=True,
    width=1000,
    height=800,
    showlegend=False
)

# Rangliste Zeugs
rangdatasort = data[data["Year"] == 2020].sort_values(by="Value", ascending=False)
rang_data = ["Country Name", "Year", "Value"]
bafig = px.bar(rangdatasort,
               x="Value",
               y="Country Name",
               orientation="h",
               color="Country Name")

bafig.update_traces(showlegend=False)

bafig.update_layout(
    title='Rangliste vom Jahr 2020',
    autosize=True,
    width=1000,
    height=1200
)

# Layout
app.layout = html.Div([

    # MAP auf Startseite
    html.H1("Frauenanteil in Managementpositionen in Prozent (senior and middle management)",
            style={'text-align': 'center'}),
    html.Div([
        dcc.Graph(
            id='chormap',
            figure=mapfig,
            responsive=True
        )
    ]),

    dbc.Tabs([

        # TAB 2 -------------------------------------------------------------------------------------
        dbc.Tab(label='Switzerland and Neighbours', children=[
            html.H2("Proportion of women in managerial positions",
                    style={'text-align': 'center'}),
            html.H4("Data for Switzerland, Austria, Germany and Italy",
                    style={'text-align': 'center'}),

            dcc.RangeSlider(
                id='scatter-slider',
                min=datafile['Year'].min(),
                max=datafile['Year'].max(),
                value=[datafile['Year'].min(), datafile['Year'].max()],
                marks={str(value): str(value) for value in datafile['Year'].unique()},
                step=None
            ),
                    html.Label("Histogram", style={'text-align': 'center'}),
                    dcc.Graph(id='hist-plot', figure=hisfig)
        ]),
        # TAB 3 -------------------------------------------------------------------------------------
        dbc.Tab(label='Ländervergleich', children=[
            html.H2("Frauenanteil in Managementpositionen in Prozent",
                    style={'text-align': 'center'}),
            
            html.Label("Wähle deine gewünschte Länder", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='lin1-dropdown',
                        options=[{'label': color, 'value': color} for color in allCountry],
                        value='Switzerland',
                        multi=True  # Mehrfachauswahl ermöglichen
                    ),
                    dcc.Graph(id='lin1-plot')
        ]),
        # TAB 4 -------------------------------------------------------------------------------------
        dbc.Tab(label='Rangliste', children=[
            html.H1("Rangliste", style={'text-align': 'center'}),
            html.Div([
                dcc.Graph(
                    # Rangliste
                    id='bafig1',
                    figure=bafig,
                )
            ])
        ]),
    ])
])

# Callbacks
@app.callback(Output('hist-plot', 'figure'), Input('scatter-slider', 'value'))
def update_histogram_plot(value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.histogram(filtered_data, x="Year", y="Value", color='GeoAreaName', barmode='group')
    return fig


@app.callback(Output('lin1-plot', 'figure'), Input('lin1-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_lin1_plot(countries, value):
    filtered_data = data[(data['Country Name'].isin(countries)) & (data['Year'] >= value[0]) & (data['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='Country Name')
    return fig


if __name__ == '__main__':
    app.run_server(debug=False, port=8015)

Dash is running on http://127.0.0.1:8015/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8015/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:05:00] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
[2023-06-28 20:05:00,342] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\wanda\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\wanda\anaconda3\lib\site-packa

127.0.0.1 - - [28/Jun/2023 20:12:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 20:12:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [4]:
# Importe
import pandas as pd
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output, callback
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

# Dateipfad
url = "https://raw.githubusercontent.com/hon3ypi/DashboardDesignFS23/main/newDataset.csv"

# Daten aus CSV-Datei laden
datafile = pd.read_csv(url)

# Daten für die ausgewählten Länder und Jahre filtern
countries = ['Switzerland', 'Germany', 'France', 'Austria', 'Italy', 'Liechtenstein']
filtComb = datafile.query('GeoAreaName in @countries')

# Diagramm für Tab 1
scatfig = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')
hisfig = px.histogram(filtComb, x="Year", y="Value", color='GeoAreaName', barmode='group')

# Web-App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Bereinigter Datensatz
data = pd.read_csv('https://raw.githubusercontent.com/monsieurmaussion/fhgr-dade/main/woman-leadership_bereinigt-neu.csv')

# Datensatz nach Jahr sortiert für animation_frame
data = data.sort_values(by="Year", ascending=True)

# Plotly choropleth map
hover_data = ["Country Name", "Year", "Value"]
mapfig = px.choropleth(data_frame=data,
                    color_continuous_scale=px.colors.sequential.Purples,
                    color="Value",
                    hover_data=hover_data,
                    locations='Country Code',
                    animation_frame='Year')

mapfig.update_layout(
    #title='Female share of employment in senior and middle management (%)',
    autosize=True, 
    width=1000, 
    height=800,
    showlegend=False)

# Rangliste Zeugs
rangdatasort = data[data["Year"] == 2020].sort_values(by="Value", ascending=False)
rang_data = ["Country Name", "Year", "Value"]
bafig = px.bar(rangdatasort,
                x="Value",
                y="Country Name",
              orientation="h",
              color="Country Name")

bafig.update_traces(showlegend=False)

bafig.update_layout(
    title='Rangliste vom Jahr 2020',
    autosize=True, 
    width=1000, 
    height=1200
)

# Layout
app.layout = html.Div([
    
    # MAP auf Startseite
     html.H1("Frauen in Managementpositionen in Prozent (senior and middle management)", style={'text-align': 'center'}),
            html.Div([
                dcc.Graph(
                    id='chormap',
                    figure=mapfig ,
                    #style={'width': '100%', 'height': '100%', 'margin': 'auto'},
                    responsive=True
                )
            ]),
    
    dbc.Tabs([

        # TAB 2 -------------------------------------------------------------------------------------
        dbc.Tab(label='Switzerland and Neighbours', children=[
            html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
            html.H4("Data for Switzerland, Austria, Germany and Italy", style={'text-align': 'center'}),

            dcc.RangeSlider(
                id='scatter-slider',
                min=datafile['Year'].min(),
                max=datafile['Year'].max(),
                value=[datafile['Year'].min(), datafile['Year'].max()],
                marks={str(value): str(value) for value in datafile['Year'].unique()},
                step=None
            ),

            dbc.Row([
                dbc.Col(width=6, children=[
                    html.Label("Histogram", style={'text-align': 'center'}),
                    dcc.Graph(id='hist-plot', figure=hisfig)
                ]),
                dbc.Col(width=6, children=[
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='line-country-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Switzerland'
                    ),
                    dcc.Graph(id='line-plot', figure=scatfig)
                ]),
            ]),
        ]),
        # TAB 3 -------------------------------------------------------------------------------------
        # Vergleich zwischen zwei Länder, Liniendiagramm als Output, Dropdownliste als Input
        dbc.Tab(label='Compare', children=[
            html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
            html.H4("Compare (only Switzerland, France, Italy, Austria & Germany)", style={'text-align': 'center'}),
            dbc.Row([
                dbc.Col(className="col-6", children=[
                    html.H4("this", style={'text-align': 'center'}),
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='lin1-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Switzerland'
                    ),
                    dcc.Graph(id='lin1-plot')
                ]),
                dbc.Col(className="col-6", children=[
                    html.H4("that", style={'text-align': 'center'}),
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='lin2-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Germany'
                    ),
                    dcc.Graph(id='lin2-plot')
                ]),
            ]),
        ]),
        # TAB 4 -------------------------------------------------------------------------------------
        dbc.Tab(label='Rangliste', children=[
            html.H1("Rangliste", style={'text-align': 'center'}),
            html.Div([
                dcc.Graph(
                   #Rangliste
                    id='bafig1',
                    figure=bafig ,
                )
            ])
        ]),
    ])
])

# Callbacks
@app.callback(Output('hist-plot', 'figure'), Input('scatter-slider', 'value'))
def update_histogram_plot(value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.histogram(filtered_data, x="Year", y="Value", color='GeoAreaName', barmode='group')
    return fig

@app.callback(Output('line-plot', 'figure'), Input('line-country-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_line_plot(color, value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName')
    
    # Farben beibehalten und nur das ausgewählte Land anzeigen
    for i, data in enumerate(fig.data):
        if data.name == color:
            fig.data[i].visible = True
        else:
            fig.data[i].visible = False
    
    return fig

@app.callback(Output('lin1-plot', 'figure'), Input('lin1-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_scatty1_plot(country, value):
    filtered_data = filtComb[(filtComb['GeoAreaName'] == country) & (filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName', color_discrete_map={"Switzerland": "red", "Germany": "black", "Italy": "green", "France": "blue", "Austria": "pink"})
    return fig

@app.callback(Output('lin2-plot', 'figure'), Input('lin2-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_scatty2_plot(country, value):
    filtered_data = filtComb[(filtComb['GeoAreaName'] == country) & (filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName', color_discrete_map={"Switzerland": "red", "Germany": "black", "Italy": "green", "France": "blue", "Austria": "pink"})
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 19:59:24] "POST /_dash-

In [ ]:
        # TAB 1 -------------------------------------------------------------------------------------
        dbc.Tab(label='Choropleth Map', children=[
            html.H1("Female share of employment in senior and middle management (%)", style={'text-align': 'center'}),
            html.Div([
                dcc.Graph(
                    id='chormap',
                    figure=mapfig ,
                    #style={'width': '100%', 'height': '100%', 'margin': 'auto'},
                    responsive=True
                )
            ])
        ]),

In [ ]:
, color_discrete_map={"Switzerland": "red", "Germany": "black", "Italy": "green", "France": "blue", "Austria": "pink"}

In [ ]:
@app.callback(Output('line-plot', 'figure'), Input('line-country-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_line_plot(countries, value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName')

    # Farben beibehalten und nur die ausgewählten Länder anzeigen
    for i, data in enumerate(fig.data):
        if data.name in countries:
            fig.data[i].visible = True
        else:
            fig.data[i].visible = False

    return fig
